In [1]:
import numpy as np

In [13]:
class Parameters():
    
    def __init__(self, parameter_values):
        self.parameters = parameter_values
        self.parent_units = set()
        
    def add_parent(self, parent):
        self.parent_units.add(parent)
    
    def get_parameters():
        return self.parameters

In [ ]:
class NeuronConnection():
    
    def __init__(self, from_unit:Neuron, to_unit:Neuron, weight):
        self.from_unit = from_unit
        self.to_unit = to_unit
        self.weight = Parameters(weight)
        self.weight.add_parent(self.from_unit)
    

In [14]:
class Neuron():
    
    def __init__(self, name):
        self.name = name
        self.energy_potential = 0
        self.connections = []
        self.activation_function = None
    
    def update(self, energy_potential):
        self.energy_potential += energy_potential
    
    def fire(self):
        
        if self.activation_function != None:
            current_energy_potential = self.activation_function(self.energy_potential)
        else:
            current_energy_potential = self.energy_potential
        
        for conn in self.connections:
            conn.to_unit.update(conn.weight.get_parameters()*current_energy_potential)
    
    def flush_energy(self):
        self.energy_potential = 0
        for conn in self.connections:
            conn.to_unit.flush_energy()
    
    def __repr__(self): # makes printing a neuron object reveal its name
        return self.name

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
sigmoid(-1000), sigmoid(0), sigmoid(1000)

/var/folders/hd/kdltsywn53jbp7lv3g1hld3w0000gn/T/ipykernel_49450/234028601.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


(0.0, 0.5, 1.0)

In [4]:

x = [Neuron('x0'), Neuron('x1'), Neuron('x2')]
z1 = [Neuron('z1_0'), Neuron('z1_1'), Neuron('z1_2')]
z2 = [Neuron('z2_0'), Neuron('z2_1'), Neuron('z2_2')]
y = Neuron('y')


In [5]:
x[0].connections = [ # w^1_01, w^1_02
    NeuronConnection(
        from_unit=x[0],
        to_unit=z1[1],
        weight=-1
    ),
    NeuronConnection(
        from_unit=x[0],
        to_unit=z1[2],
        weight=1
    )
]
# x[0].activation_function = sigmoid
x[1].connections = [ # w^1_11, w^1_12
    NeuronConnection(
        from_unit=x[1],
        to_unit=z1[1],
        weight=-2
    ),
    NeuronConnection(
        from_unit=x[1],
        to_unit=z1[2],
        weight=2
    )
]
# x[1].activation_function = sigmoid
x[2].connections = [ # w^1_21, w^1_22
    NeuronConnection(
        from_unit=x[2],
        to_unit=z1[1],
        weight=-3
    ),
    NeuronConnection(
        from_unit=x[2],
        to_unit=z1[2],
        weight=3
    )
]
# x[2].activation_function = sigmoid

In [6]:
z1[0].connections = [ # w^2_01, w^2_02
    NeuronConnection(
        from_unit=z1[0],
        to_unit=z2[1],
        weight=-1
    ),
    NeuronConnection(
        from_unit=z1[0],
        to_unit=z2[2],
        weight=1
    )
]
# z1[0].activation_function = sigmoid
z1[1].connections = [z2[1], z2[2]]
z1[1].init_parameters(Parameters([-2, 2])) # w^2_11, w^2_12
z1[1].activation_function = sigmoid
z1[2].connections = [z2[1], z2[2]]
z1[2].init_parameters(Parameters([-3, 3])) # w^2_21, w^2_22
z1[2].activation_function = sigmoid

In [7]:
z2[0].connections = [y]
z2[0].init_parameters(Parameters([-1])) # w^3_01
# z2[0].activation_function = sigmoid
z2[1].connections = [y]
z2[1].init_parameters(Parameters([2])) # w^3_11
z2[1].activation_function = sigmoid
z2[2].connections = [y]
z2[2].init_parameters(Parameters([-1.5])) # w^3_21
z2[2].activation_function = sigmoid

In [8]:
def run(X):
    
    x[0].flush_energy()
    x[1].flush_energy()
    x[2].flush_energy()
    
    x[0].energy_potential = 1
    z1[0].energy_potential = 1
    z2[0].energy_potential = 1
    
    x[1].energy_potential = X[0]
    x[2].energy_potential = X[1]
    
    x[0].fire()
    x[1].fire()
    x[2].fire()
    print(z1[0].energy_potential, z1[1].energy_potential, z1[2].energy_potential)
    
    z1[0].fire()
    z1[1].fire()
    z1[2].fire()
    print(z2[0].energy_potential, z2[1].energy_potential, z2[2].energy_potential)
    
    z2[0].fire()
    z2[1].fire()
    z2[2].fire()
    
    return y.energy_potential

In [9]:
run([1,1])

1 -6 6
1 -3.9975273768433657 3.9975273768433657


-2.436895226556018

# ^ matches problem 2

In [10]:
class Backprop:
    
    def __init__(self):
        self.__cache__ = {}
        
    def find_unit_paths(self, from_node, to_node, current_path=None, visited=None):
        '''Uses DFS to find all the connecting paths between two units'''
        if current_path is None:
            current_path = []
        if visited is None:
            visited = set()
        
        current_path.append(from_node)
        visited.add(from_node)
        
        if from_node == to_node:
            # found a path
            yield list(current_path)
        else:
            for neighbor in from_node.connections:
                if neighbor not in visited:
                    yield from self.find_unit_paths(neighbor, to_node, current_path, visited)
        
        current_path.pop()
        visited.remove(from_node)

    def build_gradient(self, from_unit, to_parameter):
        for to_unit in to_parameter.parent_units:
            paths = list(self.find_unit_paths(from_unit, to_unit))
            gradient = []
            for units in paths:
                partial = 1
                for u in units:
                    partial*=u.connection_parameters.get_parameters()
                gradient.append(partial)

In [11]:
n = Backprop().backprop(z1[0], y)

In [12]:
n

[[z1_0, z2_1, y], [z1_0, z2_2, y]]

In [18]:
for i in set([1, 2, 3]):
    print(i)

1
2
3
